<a href="https://colab.research.google.com/github/Uzmamushtaque/CSCI4962-Projects-ML-AI/blob/main/Lecture_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 7

# Today's Lecture

1. Sequence Models
2. Recurrent Neural Networks (RNNs)
3. Architecture of RNNs
3. Backpropogation through time

# Introduction

Neural networks are designed in such a way that input features are largely independent of one another. However, there are certain datatypes that have sequential dependencies. For example:

1. Time Series Data: Values on successive time stamps are closely related to one another. Every successive value cannot be considered being independent form the other because in time-series data each value is dependent on the previous one.

2. Textual Data: Usually text is treated as a bag of words until recently there is greater emphasis on the ordering of the words used for better semantic insights. It is important to construct a model that takes sequential information into account.

3. Biological Data: This comprises of sequences of symbols corresponding to amino acids that form building blocks of DNA.

# Processing Sequences

Because any Deep learning model identifies numbers (and not symbols or text) we need some way of representing these non-numeric values. Let us consider text representation as a bag of words model. Consider the sentence:

*Analytics is hardly any fun*

Let us assume we want to do some sentiment analyis using words like these. One possible solution is to create one-hot encoded vectors for each of the words. Using the English vocabulary we can have vectors of size 20000(typical number of words used) where each word's position represents the occurence of that word in a sentence.

We can use these vectors as inputs to our Feed forward network to perform sentiment analysis. In the input layer we can have one input for each word/position in the sentence. The entire dataset can have different sized sentences and using variable input sizes is impossible in a conventional neural network. Additionally, small changes in word ordering can lead to semantically different connotations. Therefore, it is important to encode word ordering more directly within the architecture of the network. 

Two important considerations for processing of sequences would be:

1. The ability to receive and process inputs in the same order as they are present in the sequence.

2. Treatment of inputs at each time-stamp in a similar manner in relation to previous history of inputs.



# RNN Model